In [111]:
#Import Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
import ta

In [112]:
class DataPreProcessing():
    def __init__(self):
        self.dataframe = None
        self.X = None
        self.y1 = None
        self.y2 = None
    def read_from_csv(self):
        df=pd.read_csv('datavn30fm.csv', index_col='Date')
        self.dataframe = df
        return df
    def preprocess(self):
        self.dataframe['RSI_value']=ta.momentum.RSIIndicator(self.dataframe['Close'], window  = 14, fillna = False).rsi()
        self.dataframe['MACD_value']=ta.trend.macd(self.dataframe['Close'], window_slow=25,window_fast=12,fillna= False)
        self.dataframe['CCI_value'] = ta.trend.cci(self.dataframe["High"],self.dataframe["Low"],self.dataframe['Close'], window=25,constant=0.015,fillna=False)
    def set_attributes_and_output(self):
        self.X = self.dataframe[['Close','RSI_value','MACD_value','CCI_value']].iloc[25:]
        self.y = self.dataframe['Close'].iloc[26:]
    def train_test_divide(self, split_at='2021-12-31'):
        return \
            self.X.loc[:split_at].iloc[:-1],self.y.loc[:split_at],self.X.loc[split_at:][:-1],self.y.loc[split_at:][1:]

In [113]:
dp = DataPreProcessing()
dp.read_from_csv()
dp.preprocess()
dp.set_attributes_and_output()

X_train,y_train, X_test,  y_test = \
    dp.train_test_divide(split_at='2021-12-31')
print('Shape of X_train: ', X_train.shape)
print('Shape of y_train: ', y_train.shape)
print('Shape of X_test: ', X_test.shape)
print('Shape of y_test: ', y_test.shape)
X_train.iloc[0]
y_train.iloc[0]

Shape of X_train:  (16014, 4)
Shape of y_train:  (16014,)
Shape of X_test:  (5375, 4)
Shape of y_test:  (5375,)


952.3

In [114]:
from sklearn.metrics import mean_squared_error,precision_score

class BaseClassRegressionAnalysis():
    def __init__(self):
        # Initialize a regressor, which will handle the LinearRegression model 
        self.regressor = LinearRegression()
    
    def fit(self, X, y):
        # The regressor learn from the training data with input X and output y
        self.regressor.fit(X, y)
        
    def predict(self, X):
        # The regressor predict the result with input X (after being trained)
        # The output has the same size as output y
        return self.regressor.predict(X)
    
    def mean_square_error(self, y_real, y_predict):
        # Compare the 2 output vectors: real output and prediction, using mean square error
        return mean_squared_error(y_real, y_predict)
    def precision_scored(self, y_real, y_predict):
        # Return the value of the precision scores
        return precision_score(y_real, y_predict,zero_division=0)

In [115]:
class LinearRegressionAnalysis(BaseClassRegressionAnalysis):
    pass
     

In [118]:
lr = LinearRegressionAnalysis()

lr.fit(X_train, y_train)
y_pred = [i for i in lr.predict(X_test)]
y_test = [x for x in y_test]
print(mean_squared_error(y_pred, y_test))
print('First 10 instances prediction :     ', np.array([i for i in y_pred[:10]]))
print('Real output of first 10 instances : ', y_test[:10])
score =0
time_pred =0
hit_points = 0
for i in range(len(y_test)):
    if y_pred[i] != 0:
        time_pred +=1
        if y_test[i] == y_pred[i]:
            score +=1
print('Score IC : ', score/time_pred)


16.757873734498332
First 10 instances prediction :      [1530.83958487 1531.44565203 1537.91218283 1537.89519029 1539.12105662
 1539.12139964 1538.61418127 1537.57145989 1535.28397176 1538.66302436]
Real output of first 10 instances :  [1531.3, 1537.7, 1537.7, 1538.9, 1538.9, 1538.4, 1537.4, 1535.2, 1538.5, 1538.4]
Score IC :  0.0


In [110]:
print(time_pred)
print(score)

5083
2352
